In [1]:
# packages needed for this tutorial
import pathlib
# math and robotics
import numpy as np
import spatialmath as sm
import quaternion
from tqdm.notebook import tqdm, trange

# plotting 
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as plt

# simulated robot content and helpers
import hrr_common
import hrr_cobot_robot

# ROS
import rospy

# set printing and plotting options    
np.set_printoptions(precision=5, suppress=True)
sns.set_theme('notebook')
%matplotlib notebook

from hr_recycler_msgs.msg import (
    AdaptiveGraspingAction, AdaptiveGraspingGoal,
    CuttingAction, CuttingGoal,
    ChangeToolAction, ChangeToolGoal,    
    FingerGraspAction, FingerGraspGoal,    
    PushOpenAction, PushOpenGoal,
    VacuumGraspingAction, VacuumGraspingGoal,
    UnscrewAction, UnscrewGoal
)
from hrr_msgs.msg import CalibrateCobotAction, CalibrateCobotGoal
from hr_recycler_msgs.msg import CuttingAction, CuttingFeedback, CuttingResult, CuttingGoal, SkillResult, CobotState, MaterialType
import actionlib
from hr_recycler_msgs.msg import ToolType

from hr_recycler_msgs.msg import PlannerAction , PlannerGoal
import hrr_common as hrr_cm

In [2]:
rospy.init_node('Restore_Defaults_2_Notebook')

In [3]:
cobot_ns = hrr_common.ros_utils.fix_prefix(hrr_common.ros_utils.get_param("/cobot_ns"))
calibration_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}calibration_action_srv_name")

In [4]:
def get_client(action_topic, ActionClass, timeout_secs=2.0):
    client = actionlib.SimpleActionClient(action_topic, ActionClass)
    client.wait_for_server(timeout=rospy.Duration(timeout_secs))
    return client
def get_result(cl):
    cl.wait_for_result()
    return cl.get_result()

def cancel(cl):
    cl.cancel_all_goals()

In [5]:
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")

pybullet build time: May  8 2021 05:48:13


[rosout] some functions are not yet compiled. Expect delays upon first call


### Open and CLose tool changer / careful

In [ ]:
cobot.open_tool_changer()

In [ ]:
cobot.close_tool_changer()

## Calibrate - Asssume Skill Server Running

In [6]:
cobot.change_tool("wsg_50_dsa")

True

In [7]:
cobot.T_B_E_robot, cobot.T_B_C_robot

(   0.5742    0.8145   -0.08267  -0.2307    
   0.7679   -0.5708   -0.2908   -0.4015    
  -0.2841    0.1035   -0.9532    0.6978    
   0         0         0         1         
,
    0.8145   -0.5742   -0.08267  -0.251     
  -0.5708   -0.7679   -0.2908   -0.4726    
   0.1035    0.2841   -0.9532    0.4648    
   0         0         0         1         
)

In [8]:
cobot.R_FT_E, sm.base.rotz(-45,"deg")

(array([[ 0.70711,  0.70711,  0.     ],
        [-0.70711,  0.70711,  0.     ],
        [ 0.     ,  0.     ,  1.     ]]),
 array([[ 0.70711,  0.70711,  0.     ],
        [-0.70711,  0.70711,  0.     ],
        [ 0.     ,  0.     ,  1.     ]]))

In [9]:
calibrate_client = get_client(calibration_action_srv_name, CalibrateCobotAction)
goal = CalibrateCobotGoal()
data_path = pathlib.Path.home() / "Documents" / "skill_recordings"
data_path.mkdir(parents=True, exist_ok=True)
tool_name = hrr_common.ros_utils.get_param(f'{cobot_ns}tool_name')
if tool_name == '':
    tool_name = 'nothing'
goal.calibration_file = str(data_path / f"{tool_name}.pkl")
goal.data_file = str(data_path / f"{tool_name}_calibration.npy")
goal.recalibrate = True
goal.scale_q4 = 0.5
goal.scale_q5 = 0.5
goal.scale_q6 = 0.5
goal

calibration_file: "/home/hrr_cobot/Documents/skill_recordings/wsg_50_dsa.pkl"
data_file: "/home/hrr_cobot/Documents/skill_recordings/wsg_50_dsa_calibration.npy"
q4_dot_des: 0.0
q5_dot_des: 0.0
q6_dot_des: 0.0
scale_q4: 0.5
scale_q5: 0.5
scale_q6: 0.5
recalibrate: True
keep_current_pose: False

In [10]:
calibrate_client.send_goal(goal)

In [ ]:
cancel(calibrate_client)

In [11]:
# get client result / feedback
get_result(calibrate_client)

bias: 
  force: 
    x: -7.537765011675344
    y: 4.791994951485458
    z: 2.203252795906889
  torque: 
    x: 0.40590758887249023
    y: -1.702375248343177
    z: 1.9558878053692894
noise: 
  force: 
    x: 0.9215505826468647
    y: 0.9533822449833155
    z: 1.6679959553801051
  torque: 
    x: 1.9003740779044949
    y: 1.3035298235350001
    z: 0.8082488494724643
tool_grav: 
  x: 1.3930597960040825
  y: -2.8496527696528946
  z: -24.55593034157012
tool_com: 
  x: 0.11985501739852081
  y: 0.043475741983541066
  z: 0.9087720695053078
calibration_file: ''

In [12]:
# plot results -> copy line to terminal
print(f"please run the following line in a separate terminal:\nrosrun hrr_cobot_robot plot_ft_calibration -p {goal.calibration_file} {goal.data_file}")

please run the following line in a separate terminal:
rosrun hrr_cobot_robot plot_ft_calibration -p /home/hrr_cobot/Documents/skill_recordings/wsg_50_dsa.pkl /home/hrr_cobot/Documents/skill_recordings/wsg_50_dsa_calibration.npy


In [60]:
cobot.update()
cobot.B_F_msr, cobot.FT_F, cobot.FT.f,cobot.FT.wrench_calib, cobot.FT.wrench, cobot.FT.offset

(array([ 13.42217,  -0.44462, -26.8418 ,  -0.36728,  -5.46946,  -0.65326]),
 array([-9.8053 ,  9.17651, 26.8418 , -3.60779, -4.1272 ,  0.65326]),
 array([-9.8053 ,  9.17651, 26.8418 ]),
 array([-9.8053 ,  9.17651, 26.8418 , -3.60779, -4.1272 ,  0.65326]),
 array([-9.8053 ,  9.17651, 26.8418 , -3.60779, -4.1272 ,  0.65326]),
 array([0., 0., 0., 0., 0., 0.]))

In [62]:
rospy.sleep(5)
np.r_[quat_rotate_vec(cobot.sns_quat, rft.T @ cobot.FT_F[0:3]), quat_rotate_vec(cobot.sns_quat, rft.T @ cobot.FT_F[3:6])]

array([ 14.3171 ,  -0.10902, -23.94141,   1.39919,  -6.39305,  -0.50055])

In [59]:
rft = cobot.R_FT_E
rft

array([[ 0.70711,  0.70711,  0.     ],
       [-0.70711,  0.70711,  0.     ],
       [ 0.     ,  0.     ,  1.     ]])

In [48]:
cobot.FT.f

array([-9.59106,  9.16846, 26.86816])

In [15]:
#This is how B_F_msr works:
def quat_rotate_vec(q1, v):
    return (q1 * np.quaternion(0, *v) * q1.conjugate()).vec
np.r_[quat_rotate_vec(cobot.sns_quat, cobot.R_FT_E.T @ cobot.FT_F[0:3]), quat_rotate_vec(cobot.sns_quat, cobot.R_FT_E.T @ cobot.FT_F[3:6])]


array([ -5.50938,  -8.81588, -23.3037 ,  11.43181,   1.23628,  -2.20266])

In [41]:
np.r_[quat_rotate_vec(cobot.sns_quat, rft.T @ cobot.FT_F[0:3]), quat_rotate_vec(cobot.sns_quat, rft.T @ cobot.FT_F[3:6])]

array([ 13.30538,  -0.30504, -26.9209 ,  -0.37332,  -5.54499,  -0.76355])

In [20]:
cobot.move_to_joint_pose(cobot.q_calib)

In [ ]:
cobot._ft_handle.offset

## Compliance

In [ ]:
cobot.init_sns_vel()

In [ ]:
cobot.reset()
cobot.set_py_hybrid_force_vel_command(B_F_des=np.r_[0.0, 0.0, -5, np.zeros(3)], 
                                          K_f=8e-4, K_t=5e-3, scale_pos=0.0, scale_rot=0.0, 
                                      wrench_dir=np.r_[1.0, 1.0, 1.0, np.ones(3)],
                                      vel_dir=np.zeros(6))

In [ ]:
T = int(10 * cobot.hz)
for t in trange(T):
    cobot.update()

In [ ]:
cobot.emergency_stop()
print(cobot.state)
cobot.reset()
print(cobot.state)